In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [ ]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base ()
Base.prepare(engine, reflect=True)

# reflect the tables


In [ ]:
engine.execute('SELECT * FROM measurement LIMIT 10').fetchall()

In [ ]:
# We can view all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
measurement = Base.classes.measurement
station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

In [ ]:
# Calculate the date 1 year ago from the last data point in the database
session.query(measurement.date).order_by(measurement.date.desc()).first()

In [ ]:
# Perform a query to retrieve the data and precipitation scores

session.query(measurement.station, func.count(measurement.date)).\
    group_by(measurement.station).\
    order_by(func.count(measurement.date).desc()).all()


results= session.query(measurement.date, measurement.prcp).\
    filter(measurement.date >= '2017-08-23').all()

# Save the query results as a Pandas DataFrame and set the index to the date column
# Sort the dataframe by date

rain_df = pd.DataFrame(results).set_index('date')
rain_df = rain_df.sort_values(by = 'date').dropna()
rain_df

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
rain_df.plot(rot = 90)
plt.xlabel("Date")
plt.ylabel("Inches")
plt.show()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
rain_df.describe()

In [ ]:
# Design a query to show how many stations are available in this dataset?
session.query(station.station).count()

In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.
session.query(measurement.station, func.count(measurement.date)).\
    group_by(measurement.station).\
    order_by(func.count(measurement.date).desc()).all()

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?
session.query(measurement.station, func.max(measurement.tobs), func.min(measurement.tobs), func.avg(measurement.tobs)).\
    filter(measurement.station == 'USC00519281').\
    group_by(measurement.station).all()

In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
temperatures = session.query(measurement.date, measurement.tobs).\
    filter(measurement.date > '2016-08-23').\
    filter(measurement.station == 'USC00519281').all()

temperatures_df = pd.DataFrame(temperatures)

temperatures_df.plot.hist(bins=12)
plt.xlabel('Temperature')
plt.ylabel('Frequency')
plt.show()